In [42]:
import tensorflow as tf
#tf.enable_eager_execution()
#from graphlib.keras_model import CarRatingPrediction
from graphlib.cryptographer import Cryptographer
from graphlib.volvo_stats import VolvoStats
from graphlib.types import Types
#from graphlib.carbinarizer import CarBinarizer
from graphlib.find_distribution import best_fit_distribution
import pandas as pd
import pandas_profiling as pp
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import keras
from keras import optimizers
from keras import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import plot_model
from keras.layers import Embedding, Dense, Input, concatenate, dot, CuDNNLSTM, RepeatVector, Dropout, LeakyReLU, Flatten, BatchNormalization, Activation
from IPython.display import Image
from datetime import datetime
import tensorflow as tf
from tensorflow import set_random_seed
import numpy as np
import seaborn as sns
from scipy.stats import norm
from graphlib.cryptographer import Cryptographer
from graphlib.volvo_stats import VolvoStats
import pickle
set_random_seed(15)
tf.__version__
from exjobb_data import *


exjobb data prepare loading...   
data loaded ....


### Define the training and test data

In [3]:
df_zip = pd.read_parquet('/home/rocket/dev/data/volvo/scb_zipstat_full.parquet')
df_zip.head(0)

,antal_inpendlare,antal_utpendlare,postnummer,man,kvinnor,forvarvs-arbetande,ej_forvarvs-arbetande,aganderatt_smahus,bostadsratt,hyresratt,ovriga_inkl_uppgift_saknas,forgymnasial,gymnasial,eftergymnasial_mindre_an_3_ar,eftergymnasial_3_ar_eller_langre_inkl_forskarutbildning,uppgift_saknas,0-6_ar,7-15_ar,16-19_ar,20-24_ar,25-44_ar,45-64_ar,65-w_ar,lag_inkomst,medellag_inkomst,medelhog_inkomst,hog_inkomst,sammanboende_med_barn,sammanboende_utan_barn,ensamstaende_med_barn,ensamstaende_utan_barn,ovriga_hushall,lag_kopkraft,medellag_kopkraft,medelhog_kopkraft,hog_kopkraft,jordbruk_skogsbruk_jakt_och_fiske,tillverkning_och_utvinning,energi_och_miljoverksamhet,byggverksamhet,handel,transport_och_magasinering,hotell-_och_restaurangverksamhet,information_och_kommunikation,finans-_och_forsakringsverksamhet,fastighetsverksamhet,foretagstjanster,offentlig_forvaltning_och_forsvar,utbildning,vard_och_omsorg_sociala_tjanster,kulturella_och_personliga_tjanster_mm,okand_verksamhet,0_barn,1_barn,2_barn,3+_barn,postal_area


In [4]:
df = pd.read_pickle('/home/rocket/dev/data/volvo/volvo_vehicles_with_zip_and_my.pickle')
df.columns

Index(['TYPECODE', 'ENGINECODE', 'SALESVERSIONCODE', 'COLOUR', 'UPHOLSTERY',
       'GEARBOXCODE', 'PACKAGES_CODES', 'OPT_CODES', 'Zip_Code', 'postnummer',
       'antal_inpendlare', 'antal_utpendlare', 'förvärvs-arbetande',
       'ej_förvärvs-arbetande', 'äganderätt/småhus', 'bostadsrätt',
       'hyresrätt', 'övriga_inkl._uppgift_saknas', 'förgymnasial', 'gymnasial',
       'eftergymnasial_mindre_än_3_år',
       'eftergymnasial_3_år_eller_längre_inkl._forskarutbildning', '0-6_år',
       '7-15_år', '16-19_år', '20-24_år', '25-44_år', '45-64_år', '65-w_år',
       'låg_inkomst', 'medellåg_inkomst', 'medelhög_inkomst', 'hög_inkomst',
       'medianinkomst', 'sammanboende_med_barn', 'sammanboende_utan_barn',
       'ensamstående_med_barn', 'ensamstående_utan_barn', 'övriga_hushåll',
       'låg_köpkraft', 'medellåg_köpkraft', 'medelhög_köpkraft',
       'hög_köpkraft', 'median_köpkraft',
       'jordbruk,_skogsbruk,_jakt_och_fiske', 'tillverkning_och_utvinning',
       'energi_och_mil

In [5]:
### USER COLUMNS
user_columns = ['antal_inpendlare', 'antal_utpendlare', 'förvärvs-arbetande',
       'ej_förvärvs-arbetande', 'äganderätt/småhus', 'bostadsrätt',
       'hyresrätt', 'övriga_inkl._uppgift_saknas', 'förgymnasial', 'gymnasial',
       'eftergymnasial_mindre_än_3_år',
       'eftergymnasial_3_år_eller_längre_inkl._forskarutbildning', '0-6_år',
       '7-15_år', '16-19_år', '20-24_år', '25-44_år', '45-64_år', '65-w_år',
       'låg_inkomst', 'medellåg_inkomst', 'medelhög_inkomst', 'hög_inkomst',
       'medianinkomst', 'sammanboende_med_barn', 'sammanboende_utan_barn',
       'ensamstående_med_barn', 'ensamstående_utan_barn', 'övriga_hushåll',
       'låg_köpkraft', 'medellåg_köpkraft', 'medelhög_köpkraft',
       'hög_köpkraft', 'median_köpkraft',
       'jordbruk,_skogsbruk,_jakt_och_fiske', 'tillverkning_och_utvinning',
       'energi_och_miljöverksamhet', 'byggverksamhet', 'handel',
       'transport_och_magasinering', 'hotell-_och_restaurangverksamhet',
       'information_och_kommunikation', 'finans-_och_försäkringsverksamhet',
       'fastighetsverksamhet', 'företagstjänster',
       'offentlig_förvaltning_och_försvar', 'utbildning',
       'vård_och_omsorg,_sociala_tjänster',
       'kulturella_och_personliga_tjänster_m.m.', 'okänd_verksamhet', '0_barn',
       '1_barn', '2_barn', '3+_barn']

vehicle_columns = ['TYPECODE', 'SALESVERSIONCODE', 'GEARBOXCODE', 'COLOUR', 'UPHOLSTERY', 'ENGINECODE','OPT_CODES']

In [6]:
df[user_columns].head(0)

,antal_inpendlare,antal_utpendlare,förvärvs-arbetande,ej_förvärvs-arbetande,äganderätt/småhus,bostadsrätt,hyresrätt,övriga_inkl._uppgift_saknas,förgymnasial,gymnasial,eftergymnasial_mindre_än_3_år,eftergymnasial_3_år_eller_längre_inkl._forskarutbildning,0-6_år,7-15_år,16-19_år,20-24_år,25-44_år,45-64_år,65-w_år,låg_inkomst,medellåg_inkomst,medelhög_inkomst,hög_inkomst,medianinkomst,sammanboende_med_barn,sammanboende_utan_barn,ensamstående_med_barn,ensamstående_utan_barn,övriga_hushåll,låg_köpkraft,medellåg_köpkraft,medelhög_köpkraft,hög_köpkraft,median_köpkraft,"jordbruk,_skogsbruk,_jakt_och_fiske",tillverkning_och_utvinning,energi_och_miljöverksamhet,byggverksamhet,handel,transport_och_magasinering,hotell-_och_restaurangverksamhet,information_och_kommunikation,finans-_och_försäkringsverksamhet,fastighetsverksamhet,företagstjänster,offentlig_förvaltning_och_försvar,utbildning,"vård_och_omsorg,_sociala_tjänster",kulturella_och_personliga_tjänster_m.m.,okänd_verksamhet,0_barn,1_barn,2_barn,3+_barn


In [7]:
df.MODEL_YEAR = pd.to_numeric(df.MODEL_YEAR)

In [8]:
df = df.dropna(subset=user_columns)
df[df[user_columns].isnull().any(axis=1)]

,TYPECODE,ENGINECODE,SALESVERSIONCODE,COLOUR,UPHOLSTERY,GEARBOXCODE,PACKAGES_CODES,OPT_CODES,Zip_Code,postnummer,antal_inpendlare,antal_utpendlare,förvärvs-arbetande,ej_förvärvs-arbetande,äganderätt/småhus,bostadsrätt,hyresrätt,övriga_inkl._uppgift_saknas,förgymnasial,gymnasial,eftergymnasial_mindre_än_3_år,eftergymnasial_3_år_eller_längre_inkl._forskarutbildning,0-6_år,7-15_år,16-19_år,20-24_år,25-44_år,45-64_år,65-w_år,låg_inkomst,medellåg_inkomst,medelhög_inkomst,hög_inkomst,medianinkomst,sammanboende_med_barn,sammanboende_utan_barn,ensamstående_med_barn,ensamstående_utan_barn,övriga_hushåll,låg_köpkraft,medellåg_köpkraft,medelhög_köpkraft,hög_köpkraft,median_köpkraft,"jordbruk,_skogsbruk,_jakt_och_fiske",tillverkning_och_utvinning,energi_och_miljöverksamhet,byggverksamhet,handel,transport_och_magasinering,hotell-_och_restaurangverksamhet,information_och_kommunikation,finans-_och_försäkringsverksamhet,fastighetsverksamhet,företagstjänster,offentlig_förvaltning_och_försvar,utbildning,"vård_och_omsorg,_sociala_tjänster",kulturella_och_personliga_tjänster_m.m.,okänd_verksamhet,0_barn,1_barn,2_barn,3+_barn,MODEL_YEAR


In [9]:
isinstance(df[df['OPT_CODES'].isnull()].head(1)['OPT_CODES'].values[0],type(np.nan))

False

In [10]:
def remove_nan(series):
    if isinstance(series,type(None)):
        return np.nan
    return [x for x in series if str(x) != 'nan']
df['OPT_CODES'] = df['OPT_CODES'].progress_apply(lambda x: remove_nan(x))

In [11]:
df = df.dropna(subset=['OPT_CODES'])

In [13]:
df.head(0)

,TYPECODE,ENGINECODE,SALESVERSIONCODE,COLOUR,UPHOLSTERY,GEARBOXCODE,PACKAGES_CODES,OPT_CODES,Zip_Code,postnummer,antal_inpendlare,antal_utpendlare,förvärvs-arbetande,ej_förvärvs-arbetande,äganderätt/småhus,bostadsrätt,hyresrätt,övriga_inkl._uppgift_saknas,förgymnasial,gymnasial,eftergymnasial_mindre_än_3_år,eftergymnasial_3_år_eller_längre_inkl._forskarutbildning,0-6_år,7-15_år,16-19_år,20-24_år,25-44_år,45-64_år,65-w_år,låg_inkomst,medellåg_inkomst,medelhög_inkomst,hög_inkomst,medianinkomst,sammanboende_med_barn,sammanboende_utan_barn,ensamstående_med_barn,ensamstående_utan_barn,övriga_hushåll,låg_köpkraft,medellåg_köpkraft,medelhög_köpkraft,hög_köpkraft,median_köpkraft,"jordbruk,_skogsbruk,_jakt_och_fiske",tillverkning_och_utvinning,energi_och_miljöverksamhet,byggverksamhet,handel,transport_och_magasinering,hotell-_och_restaurangverksamhet,information_och_kommunikation,finans-_och_försäkringsverksamhet,fastighetsverksamhet,företagstjänster,offentlig_förvaltning_och_försvar,utbildning,"vård_och_omsorg,_sociala_tjänster",kulturella_och_personliga_tjänster_m.m.,okänd_verksamhet,0_barn,1_barn,2_barn,3+_barn,MODEL_YEAR


In [14]:
import pandas as pd
from graphlib.types import Types

class VolvoStats:
    def __init__(self, df, debug=False, group_by='default'):
        self.debug = debug
        assert df is not None, "You must provide a dataframe if you don't want to reload it"
        self.volvo_df = df

        if group_by == 'default':
            self.group_by = self.volvo_df.MODEL_YEAR
        else: 
            self.group_by = group_by(df)

        self.agg_stats = {
            'counts': VolvoStats.top_k_values,
            'percentage': VolvoStats.top_k_count_percentage
        }

        self.mappings = {
            'ENGINECODE': Types.ENGINE,
            'TYPECODE': Types.CAR_TYPE,
            'SALESVERSIONCODE': Types.SALES_VERSION,
            'GEARBOXCODE': Types.GEARBOX,
            'COLOUR': Types.COL,
            'UPHOLSTERY': Types.UPH,
            'OPT_CODES': Types.OPT,
        }

        for key in self.mappings.keys():
            assert key in self.volvo_df.columns, "The dataframe provided does not contain column {}".format(
                key)
        self.cache = {}

        self.agg = {}
        for key in self.mappings:
            if key is 'OPT_CODES':
                self.agg[key] = {
                    'counts':VolvoStats.tok_k_values_list,
                    'percentage':VolvoStats.top_k_percentage_list
                }
            else:
                self.agg[key] = self.agg_stats

        self._init_statistics()

    def debug_print(self, *msg):
        if self.debug:
            print(msg)

    def _init_statistics(self):
        self.volvo_agg_df = self.volvo_df.groupby(self.group_by).agg(self.agg)
        for year in self.volvo_agg_df.index:
            for code, typ in self.mappings.items():
                stats = self.volvo_agg_df.loc[year][code]
                if not year in self.cache:
                    self.cache[year] = {}
                self.cache[year][typ] = dict(
                    zip(stats['counts'], stats['percentage']))

    @staticmethod
    def top_k_count(series):
        return ",".join(map(lambda x: str(x).strip(), series.value_counts())).replace(',,', ',')

    @staticmethod
    def top_k_values(series):
        return list(map(lambda x: x, series.value_counts().index))

    @staticmethod
    def top_k_count_percentage(series):
        total = series.value_counts().sum()
        values = [(v/total)
                  for _, v in series.value_counts().to_dict().items()]
        return values
    
    def top_k_list(series_w_list):
        series_dict= series_w_list.to_dict()
        items = []
        for list_items in series_dict.values():
            #print('kv',list_items)
            if isinstance(list_items,list):
                items += list_items                
        return Counter(items)

    def tok_k_values_list(series):
        return list(map(lambda x: x[0], top_k(series).most_common()))

    def top_k_percentage_list(series):
        series_length=len(series.to_dict().keys())
        return list(map(lambda x: x[1]/series_length, top_k(series).most_common()))

    @property
    def stats(self):
        return self.cache


In [29]:
from collections import Counter
import operator

def top_k(series_w_list):
    series_dict= series_w_list.to_dict()
    items = []
    for list_items in series_dict.values():
        #print('kv',list_items)
        if isinstance(list_items,list):
            items += list_items                
    return Counter(items)

def tok_k_values(series):
    return list(map(lambda x: x[0], top_k(series).most_common()))

def top_k_percentage(series):
    series_length=len(series.to_dict().keys())
    return list(map(lambda x: x[1]/series_length, top_k(series).most_common()))

    
#df.groupby(df.MODEL_YEAR).agg({'OPT_CODES':{'values':tok_k_values, 'percentage':top_k_percentage}})

In [30]:
stats = VolvoStats(df)
with open('VolvoStats', 'wb') as f:
    pickle.dump(stats,f)

In [31]:
import operator
from collections import Counter
from graphlib.types import Types

class Cryptographer:
    
    DISTRIBUTION_MOCKUP = {
        '2008': {
            Types.ENGINE: {
                'A1': 0.13756074,
                'A2': 0.12775981,
                'A3': 0.13522137,
                'A4': 0.14571605,
                'A5': 0.1719328,
                'A6': 0.14982322,
                'A7': 0.13198601
            }
        },
        '2019': {
            Types.ENGINE: {
                'B1': 0.26371891,
                'B2': 0.19470216,
                'B3': 0.1293187,
                'B4': 0.3499376,
                'B5': 0.06232263,
            }
        }
    }
    
    def __init__(self, distributions = None):
        if distributions:
            self.distributions = distributions
        else:
            self.distributions = Cryptographer.DISTRIBUTION_MOCKUP

            
    # Returns the distribution of a specific year and type. For every call, the 
    # distribution will be sorted and restored in 'self.distributions'.

    def get_dist(self, year, typ):
        if not 'sorted' in self.distributions[year][typ]:
            sorted_d = sorted(self.distributions[year][typ].items(), key=operator.itemgetter(1))
            self.distributions[year][typ] = dict(sorted_d)
        
        dist = np.array(list(self.distributions[year][typ].values()))
        keys = np.array(list(self.distributions[year][typ].keys()))
        
        return dist, keys
    
    # Input:
    #   'form_year': e.g. '2008'
    #   'typ': a graphlib.Types, e.g. Types.ENGINE
    #   'comp_code': a component code, e.g. 'A8'
    # Return:
    #   The component's (type, comp_code) distribution value in year 'from_year'

    def encode_dist(self, from_year, typ, comp_code):
        if isinstance(comp_code,list):
            dist_values = [self.distributions[from_year][typ][code] for code in comp_code]
            return dist_values
        dist_value = self.distributions[from_year][typ][comp_code]
        
        return dist_value
    
    # Input: 
    #   'to_year': e.g. '2019'
    #   'typ': a type from graphlib.Types
    #   'dist_value': the distribution value for the component that is going to be mapped
    # Return:
    #   A component, e.g. (Types.ENGINE, 'A8')

    def decode_dist(self, to_year, typ, dist_value, rounds=1,debug=False,filterer=None):

        if isinstance(dist_value,list):
            if len(dist_value) == 0:
                return []
            return [self.decode_dist(to_year,typ,v,filterer=filterer) for v in dist_value]
        predictions = []
        for _ in range(rounds):

            # Get distribution of year and type
            to_dists, to_keys = self.get_dist(to_year, typ)
            #print('1',to_dists,to_keys,dist_value)
            
            if not filterer is None:
                valids = filterer(typ,to_keys)
                result = list(filter(lambda x: x[1] in valids,zip(to_dists,to_keys)))
                #print('FILTERED:',result)
                to_dists,to_keys = zip(*result)
                to_dists=np.array(to_dists)
                to_keys=np.array(to_keys)
                
                #print('filtered',to_dists,to_keys,dist_value)
                
            # 'dist_value' is the new 100% of 'to_dists'
            diff = to_dists - dist_value
            min_idx = np.argmin(np.abs(diff))
            #print('2',diff)
            #this is the special case where we just map 1-1 to the distribution
            # based on the distribution in one year, what it represents in another year.
            # this also means that if we translate something in the same year, we will get the same thing back
            return (typ,to_keys[min_idx])
            d_n = np.append(to_dists[:min_idx], [dist_value])
            
            # Slicing to exclude items that is not directly over dist_value or under.
            # E.g. if dist_value = 0.11 and to_dists = [0.05, 0.1, 0,2, 0.5], the 'new_values'
            # includes [0.05, 0.1, 0.2] as [0.05, 0.1, 0.11] from which
            # we do a distance distribution into [0.05, 0.05, 0.01] (distances from each other with idx=0 as base).
            # This will (from here) go into a prob. distribution of [0.45454545, 0.45454545, 0.09090909]
            distances = d_n.reshape(-1,1) - d_n
            d_n[1:] = np.diag(distances, k = -1)
            pd = d_n / np.sum(d_n)
            
            # Now, we pick a random index over this distribution
            p_idx = np.random.choice(len(pd), 1, p=pd)
            
            # Return the key of p_idx as a component
            predictions+=[to_keys[p_idx][0]]

        counted = Counter(predictions)
        if debug:
            print(counted)

        most_common_component,_ = counted.most_common(1)[0]

        return (typ, most_common_component)


In [32]:
### Data pre process
cp = Cryptographer(stats.stats)
#stats.stats

In [33]:

import numpy as np
import cvxpy as cvp
import cvxopt
import re
from functools import reduce
import pickle
class SolverData:
    def __init__(self):
        self.all_pno_components = {
            'cartypes':set([]),
            'salesversions':set([]),
            'engines':set([]),
            'gearboxes':set([]),
            'uph':set([]),
            'col':set([])
        }
        self.solver_data = pickle.load(open("/home/rocket/dev/jupyter/solver_data.pkl", "rb"))
        
        self.pno12s = []
        self.colors=[]
        self.uphs=[]
        self.opts=[]
        self.pno12s_ct={}
        for ct, values in self.solver_data['css'].items():
            self.pno12s+=list(filter(lambda x: 'PNO' in x[0],values.c2i.items()))
            self.colors+=list(filter(lambda x: 'COL' in x[0],values.c2i.items()))
            self.uphs+=list(filter(lambda x: 'UPH' in x[0],values.c2i.items()))
            self.opts+=list(filter(lambda x: 'OPT' in x[0],values.c2i.items()))


        self.uphs = list(map(lambda x: ('UPH',x[0][-6:]),self.uphs))
        self.colors = list(map(lambda x: ('COL',x[0][-5:]),self.colors))
        self.opts = list(map(lambda x: ('OPT',x[0][-6:]),self.opts))
        for pno12 in self.pno12s:
            _,ct,eng,sv,gb = re.findall(r'(..PNO12..)(...)_(..)_(..)_(.)',pno12[0])[0]
            self.all_pno_components['cartypes'].add(ct)
            self.all_pno_components['salesversions'].add(sv)
            self.all_pno_components['engines'].add(eng)
            self.all_pno_components['gearboxes'].add(gb)

        self.total_length=0
        self.total_length += reduce(lambda x,y:x+y,[len(x) for x in self.all_pno_components.values() ])
        self.total_length
        #print('pno12s', self.one_hot_encoder.classes_)
        #self.one_hot_encoder.fit()
        self._build_matrix()
        
    def _build_matrix(self):
        self.matrix = np.zeros((len(self.pno12s),self.total_length),dtype=np.int8)
        
        self.ct_index=list(self.all_pno_components['cartypes'])
        self.sv_index=list(self.all_pno_components['salesversions'])
        self.eng_index=list(self.all_pno_components['engines'])
        self.gb_index=list(self.all_pno_components['gearboxes'])

        for idx in range(len(self.pno12s)):

            _,ct,eng,sv,gb = re.findall(r'(..PNO12..)(...)_(..)_(..)_(.)',self.pno12s[idx][0])[0]
            ct_i = self.ct_index.index(ct)
            sv_i = len(self.ct_index)+self.sv_index.index(sv)
            eng_i = len(self.ct_index)+len(self.sv_index)+self.eng_index.index(eng)
            gb_i = len(self.ct_index)+len(self.sv_index)+len(self.eng_index)+self.gb_index.index(gb)
            self.matrix[idx,ct_i] = 1
            self.matrix[idx,sv_i] = 1
            self.matrix[idx,eng_i] = 1
            self.matrix[idx,gb_i] = 1
            
    def find_nearest_pno12(self,ct,sv,eng,gb):
        ct_i = self.ct_index.index(ct)
        sv_i = len(self.ct_index)+self.sv_index.index(sv)
        eng_i = len(self.ct_index)+len(self.sv_index)+self.eng_index.index(eng)
        gb_i = len(self.ct_index)+len(self.sv_index)+len(self.eng_index)+self.gb_index.index(gb)
        predicted=np.zeros(self.matrix.shape[1])
        predicted[ct_i] = 1
        predicted[sv_i] = 1
        predicted[eng_i] = 1
        predicted[gb_i] = 1
        row = np.sum(np.absolute(np.subtract(self.matrix,predicted)),axis=1).argmin()
        return self.pno12s[row]
    
    def filter_values_by_pno_type(self, ct,sv,eng,gb,typ,values):
        c2i = self.solver_data['css'][ct].c2i
        #print('ALL',list(filter(lambda x: typ in x[0], c2i.items())))
        theoretical_values = [(f'__{typ}__2019{ct}{eng}{sv}{gb}{v[0]}',v[1]) for v in values]
            
        print(f'theoretircal {theoretical_values}')
        real_values = [val for val in theoretical_values if val[0] in map(lambda x: x[0], c2i.items())]
        print(f'real_values {real_values}')
        
        return real_values

    def filter_values_by_car_type(self,ct,typ,values):
        c2i = self.solver_data['css'][ct].c2i
        #print(c2i)
        all_values = list(filter(lambda x: typ in x, c2i.keys()))
        #print('filter_values_by_car_type',typ,all_values)
        if typ is 'COL':
            return list(set(map(lambda x: x[-5:],all_values)))
        else:
            return list(set(map(lambda x: x[-6:],all_values)))
        
    
    def get_onehot_pno12_for_ct(self,ct,sv,eng,gb):
        pno12, idx = self.find_nearest_pno12(ct,sv,eng,gb)
        #search
        for ct, values in self.solver_data['css'].items():
            i2c  = values['i2c']
            if i2c[idx] == pno12:
                #match
                pno12s = list(filter(lambda x: 'PNO12' in x[0],values.c2i.items()))
                onehot = np.zeros(len(pno12s))
                onehot[idx] = 1
                return onehot
            

    def get_onehot_for_ct(self,ct,typ,value):
        c2i = self.solver_data['css'][ct].c2i
        
        typ_filtered = list(filter(lambda x: typ in x[0],c2i.items()))
        one_hot = np.zeros(len(typ_filtered))
        index_typ_mapping = {}
        
        for col, idx in typ_filtered:
            position = typ_filtered.index((col,idx))
            index_typ_mapping[position] = idx
           # if value 
    
    def invert_components_vec(self,ct,comps):
        pass
    
    def build_components_vec(self,ct,sv,eng,gb, cols, uphs, opts,debug=False):
        """
            cols = [('71200',0.006),...]
            uhps = [('RA00000',0.231)]
            opts = [('079000',0.231)]
            
            
        """
        pno12 = self.find_nearest_pno12(ct,sv,eng,gb)
        #print('pno12:',pno12)
        _,ct,eng,sv,gb = re.findall(r'(..PNO12..)(...)_(..)_(..)_(.)',pno12[0])[0]
        #print(ct,eng,sv,gb)
        c2i = self.solver_data['css'][ct].c2i
        #print(c2i)
        carcomps = np.zeros(len(c2i))
        carcomps[pno12[1]] = 1
        cols = self.filter_values_by_pno_type(ct,sv,eng,gb,'COL',cols)
        uph = self.filter_values_by_pno_type(ct,sv,eng,gb,'UPH',uphs)
        opts = self.filter_values_by_pno_type(ct,sv,eng,gb,'OPT',opts)
        
        for item, w in cols+uph+opts:
            if debug:
                print('=>',item,c2i[item],w)
            carcomps[c2i[item]] = w

        return ct,carcomps
        
    def solve(self,ct,sv,eng,gb,cols,uphs,opts,debug=False):
        if debug:
            print(ct,sv,eng,gb,cols,uphs,opts)
        ct,w = self.build_components_vec(ct,sv,eng,gb,cols,uphs,opts,debug)
        #return self.solver_data['css'][ct].solve(w,lambda w,x: -(cp.sum(cp.abs(w-x))))
        return self.solver_data['css'][ct].solve(w,lambda w,x: w@x)
        
    def invert_onehot_pno12_for_ct(self,ct,one_hot):
        argmax = one_hot.argmax()
        return self.solver_data['css'][ct]['i2c'][argmax]
    
pno12_solver = SolverData()


In [34]:
list(filter(lambda x: '000205' in x,pno12_solver.solver_data['css']['525'].c2i.keys()))

[]

In [35]:
for key,typ in stats.mappings.items():
    df[key+'_dist'] = df.progress_apply(lambda x: cp.encode_dist(x.MODEL_YEAR,typ,x[key]), axis=1)

In [36]:
pno_keys = ['CAR_TYPE','SALES_VERSION','ENGINE','GEARBOX']
def translate_row(x):
    pno_12 ={}
    for key,typ in stats.mappings.items():
        if not typ in pno_keys:
            continue
        if key is 'OPT_CODES':
            pno_12[typ] = list(map(lambda y: y[1],cp.decode_dist(2019,'OPT', x['OPT_CODES_dist'])))     
        else:
            pno_12[typ] = cp.decode_dist(2019,typ, x[f'{key}_dist'])[1]
    #print(pno_12)
    def filter_by_typ(typ,values):
        filtered= pno12_solver.filter_values_by_car_type(pno_12['CAR_TYPE'],typ,values)
        return filtered
    
    nearest = pno12_solver.find_nearest_pno12(pno_12['CAR_TYPE'],pno_12['SALES_VERSION'],pno_12['ENGINE'],pno_12['GEARBOX'])[0]
    #print('nearest',nearest)
    _,ct,eng,sv,gb = re.findall(r'(..PNO12..)(...)_(..)_(..)_(.)',nearest)[0]
    #print(ct,eng,sv,gb)
    pno_12['CAR_TYPE'] = ct
    pno_12['ENGINE'] = eng
    pno_12['GEARBOX'] = gb
    pno_12['SALES_VERSION'] = sv
    
    for key,typ in stats.mappings.items():
        if typ in pno_keys:
            continue
        if key is 'OPT_CODES':
            pno_12[typ] = list(map(lambda y: y[1],cp.decode_dist(2019,'OPT', x['OPT_CODES_dist'],filterer=filter_by_typ)))    
        else:
            pno_12[typ] = cp.decode_dist(2019,typ, x[f'{key}_dist'],filterer=filter_by_typ)[1]
    
    #print(pno_12)
    #filter_values_by_pno_type(self, ct,sv,eng,gb,typ,values)
    #print(pno12_solver.filter_values_by_pno_type(pno_12['CAR_TYPE'],pno_12['SALES_VERSION'],pno_12['ENGINE'],pno_12['GEARBOX'],'COL',[pno_12['OPT']]))
    #find_nearest_pno12(self,ct,sv,eng,gb):
    return nearest, pno_12['COL'], pno_12['UPH'], pno_12['OPT']
df['WORKING_PNO34'] = df.progress_apply(lambda x: translate_row(x),axis=1)

In [37]:
def pno12_to_dict(pno12,typ):
    _,ct,eng,sv,gb = re.findall(r'(..PNO12..)(...)_(..)_(..)_(.)',pno12[0])[0]
    if typ is 'CAR_TYPE':
        return ct
    if typ is 'ENGINE':
        return eng
    if typ is 'SALES_VERSION':
        return sv
    if typ is 'GEARBOX':
        return gb
    if typ is 'COL':
        return pno12[1]
    if typ is 'UPH':
        return pno12[2]
    if typ is 'OPT':
        return pno12[3]
    

for key,typ in stats.mappings.items():
    print(f'running {key}')
    df[f'{key}_translated'] = df['WORKING_PNO34'].progress_apply(lambda x: pno12_to_dict(x,typ))


running ENGINECODE



running TYPECODE



running SALESVERSIONCODE



running GEARBOXCODE



running COLOUR



running UPHOLSTERY



running OPT_CODES


In [38]:
df[['WORKING_PNO34','TYPECODE_translated','ENGINECODE_translated','SALESVERSIONCODE_translated','GEARBOXCODE_translated']].sample(0)

,WORKING_PNO34,TYPECODE_translated,ENGINECODE_translated,SALESVERSIONCODE_translated,GEARBOXCODE_translated


In [39]:
df[map(lambda x: x,stats.mappings.keys())].head(0)

,ENGINECODE,TYPECODE,SALESVERSIONCODE,GEARBOXCODE,COLOUR,UPHOLSTERY,OPT_CODES


In [40]:
df[map(lambda x: x+'_translated',stats.mappings.keys())].head(0)

,ENGINECODE_translated,TYPECODE_translated,SALESVERSIONCODE_translated,GEARBOXCODE_translated,COLOUR_translated,UPHOLSTERY_translated,OPT_CODES_translated


In [43]:
cb = CarBinarizer()
all_items = list(map(lambda x: ('CAR_TYPE',x) ,pno12_solver.all_pno_components['cartypes']))+\
list(map(lambda x: ('SALES_VERSION',x) ,pno12_solver.all_pno_components['salesversions']))+\
list(map(lambda x: ('ENGINE',x) ,pno12_solver.all_pno_components['engines']))+\
list(map(lambda x: ('GEARBOX',x) ,pno12_solver.all_pno_components['gearboxes']))+\
pno12_solver.uphs+pno12_solver.colors+pno12_solver.opts

In [44]:
cb.fit(all_items)
cb


CarBinarizer(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [46]:
with open("CarBinarizer", "wb") as f:
    pickle.dump(cb, f)
with open("Stats", "wb") as f:
    pickle.dump(cb, f)

In [47]:
cb.types_binarizer

{'CAR_TYPE': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'SALES_VERSION': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'ENGINE': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'GEARBOX': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'UPH': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'COL': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'OPT': MultiLabelBinarizer(classes=None, sparse_output=False)}

In [48]:
#ONE HOT

def map_to_onehot(x,typ):
    return cb.types_binarizer[typ].transform([x])


for df_typ,typ in tqdm_notebook(stats.mappings.items()):
    df[f'{df_typ}_onehot'] = df[f'{df_typ}_translated'].progress_apply(lambda x: map_to_onehot(x,typ))

In [49]:
def remap(series):
    return np.array(list(map(lambda x: x[0],series.values)))
def create_train_test_split(use_data,test_size=0.2):
    X_train,X_test, y_typecode_train,y_typecode_test,y_sv_train,y_sv_test,y_eng_train,y_eng_test,y_gb_train,y_gb_test,\
        y_uph_train,y_uph_test,y_col_train,y_col_test,y_opt_train,y_opt_test= train_test_split(use_data,\
        remap(use_data['TYPECODE_onehot']),\
        remap(use_data['SALESVERSIONCODE_onehot']),\
        remap(use_data['ENGINECODE_onehot']),\
        remap(use_data['GEARBOXCODE_onehot']),\
        remap(use_data['UPHOLSTERY_onehot']),\
        remap(use_data['COLOUR_onehot']),
        remap(use_data['OPT_CODES_onehot']),test_size=test_size)
    return X_train,X_test, y_typecode_train,y_typecode_test,y_sv_train,y_sv_test,y_eng_train,y_eng_test,y_gb_train,y_gb_test,y_uph_train,y_uph_test,y_col_train,y_col_test,y_opt_train,y_opt_test
print(df.shape)
X_train,X_test, y_typecode_train,y_typecode_test,y_sv_train,y_sv_test,y_eng_train,y_eng_test,y_gb_train,y_gb_test,y_uph_train,y_uph_test,y_col_train,y_col_test,y_opt_train,y_opt_test= create_train_test_split(df,test_size=0.2)
X_train,X_dev, y_typecode_train,y_typecode_dev,y_sv_train,y_sv_dev,y_eng_train,y_eng_dev,y_gb_train,y_gb_dev,y_uph_dev,y_uph_dev,y_col_train,y_col_dev,y_opt_train,y_opt_dev= create_train_test_split(X_train,test_size=0.2)

X_train['TYPECODE_translated'].value_counts()
print(X_train.shape)
#X_train, y_train = merged[user_columns], mlb.transform(df_train['TYPED_CAR'])

(46674, 87)
(29871, 87)


In [53]:
X_train.to_pickle('train.pickle')

In [52]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
for key in stats.mappings.keys():
    if 'OPT' in key:
        continue
    X_res, y_res = ros.fit_resample(X_train, X_train[f'{key}_translated'])
    resampled = pd.DataFrame(X_res,columns=X_train.columns)
    #print(key,  resampled[f'{key}_translated'].value_counts())
    #print('file',f'{key}_train.pickle')
    resampled.to_pickle(f'{key}_train.pickle')


In [50]:
dev = X_dev.to_pickle('dev.pickle')
test = X_test.to_pickle('test.pickle')